In [1]:
import numpy as np
import cv2

from scipy.interpolate import interp2d

from skimage import io


In [8]:
fileaddr = 'F:/Folder_name/'
filename = 'File_name'

image = np.float32(io.imread(fileaddr + filename + '.tif'))

scale = 6

fx = scale
fy = scale

pad_image = np.zeros([3*image.shape[0], 3*image.shape[1]])

pad_image[0:image.shape[0], 0:image.shape[1]] = image
pad_image[0:image.shape[0], image.shape[1]:2*image.shape[1]] = image
pad_image[0:image.shape[0], 2*image.shape[1]:3*image.shape[1]] = image
pad_image[image.shape[0]:2*image.shape[0], 0:image.shape[1]] = image
pad_image[image.shape[0]:2*image.shape[0], image.shape[1]:2*image.shape[1]] = image
pad_image[image.shape[0]:2*image.shape[0], 2*image.shape[1]:3*image.shape[1]] = image
pad_image[2*image.shape[0]:3*image.shape[0], 0:image.shape[1]] = image
pad_image[2*image.shape[0]:3*image.shape[0], image.shape[1]:2*image.shape[1]] = image
pad_image[2*image.shape[0]:3*image.shape[0], 2*image.shape[1]:3*image.shape[1]] = image



In [9]:
f_pad_image = np.fft.fftshift(np.fft.fft2(pad_image, s=pad_image.shape))

x = np.arange(f_pad_image.shape[1])
y = np.arange(f_pad_image.shape[0])

interp = interp2d(y, x, f_pad_image.real)

x_new = np.arange(fx*f_pad_image.shape[1])
y_new = np.arange(fy*f_pad_image.shape[0])

new_real = interp(y_new, x_new)


interp = interp2d(y, x, f_pad_image.imag)

new_imag = interp(y_new, x_new)

f_new_pad = new_real + 1j*new_imag

new_pad = np.abs(np.fft.ifft2(f_new_pad, s=f_new_pad.shape))

new_pad_rescaled = (new_pad-np.mean(new_pad))/np.std(new_pad)*np.std(pad_image)+np.mean(pad_image)

cropped_new_pad = new_pad_rescaled[fx*image.shape[0]:2*fx*image.shape[0], fx*image.shape[1]:2*fx*image.shape[1]]

io.imsave(fileaddr + filename + '_fourier_' + str(fx) + 'x.tif', np.float32(cropped_new_pad))